In [13]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

ks = pd.read_csv('data/ks-projects-201801.csv',
                parse_dates=['deadline','launched'])
ks.head(1)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95


In [14]:
# Add outcome column, "successful" == 1, others are 0
ks = ks.query('state != "Live"')
ks = ks.assign(outcome=(ks['state'] == 'successful').astype(int))
ks.head(3)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,outcome
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95,0
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00,0
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00,0


In [15]:
# Timestamp Features
ks = ks.assign(hour=ks.launched.dt.hour,
              day=ks.launched.dt.day,
              month=ks.launched.dt.month,
              year=ks.launched.dt.year)
ks.head(1)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,outcome,hour,day,month,year
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95,0,12,11,8,2015


In [16]:
# Label encoding
cat_features = ['category','currency','country']
encoder = LabelEncoder()
encoded = ks[cat_features].apply(encoder.fit_transform)
encoded.head(3)

,category,currency,country
0,108,5,9
1,93,13,22
2,93,13,22


In [17]:
data_cols = ['goal','hour','day','month','year','outcome']
baseline_data = ks[data_cols].join(encoded)
baseline_data

,goal,hour,day,month,year,outcome,category,currency,country
0,1000.0,12,11,8,2015,0,108,5,9
1,30000.0,4,2,9,2017,0,93,13,22
2,45000.0,0,12,1,2013,0,93,13,22
3,5000.0,3,17,3,2012,0,90,13,22
4,19500.0,8,4,7,2015,0,55,13,22
...,...,...,...,...,...,...,...,...,...
378656,50000.0,2,17,9,2014,0,39,13,22
378657,1500.0,3,22,6,2011,0,93,13,22
378658,15000.0,19,1,7,2010,0,93,13,22
378659,15000.0,18,13,1,2016,0,138,13,22


In [18]:
# Defining functions that will help us test our encodings
import lightgbm as lgb
from sklearn import metrics

def get_data_splits(dataframe, valid_fraction=0.1):
    valid_fraction = 0.1
    valid_size = int(len(dataframe) * valid_fraction)
    
    train = dataframe[:-valid_size * 2]
    # valid size == test size, last two sections of the data
    valid = dataframe[-valid_size * 2:-valid_size]
    test = dataframe[-valid_size:]
    
    return train, valid, test

def train_model(train, valid):
    feature_cols = train.columns.drop('outcome')
    
    dtrain = lgb.Dataset(train[feature_cols], label=train['outcome'])
    dvalid = lgb.Dataset(valid[feature_cols], label=valid['outcome'])
    
    param = {'num_leaves':64, 'objective':'binary',
            'metric':'auc', 'seed':7}
    print("Training model!")
    bst = lgb.train(param, dtrain, num_boost_round=1000, valid_sets=[dvalid],
                   early_stopping_rounds=10, verbose_eval=False)
    
    valid_pred = bst.predict(valid[feature_cols])
    valid_score = metrics.roc_auc_score(valid['outcome'], valid_pred)
    print(f"Validation AUC score: {valid_score:.4f}")
    return bst

In [19]:
# Training on the baseline data
train, valid, _ = get_data_splits(baseline_data)
bst = train_model(train, valid)

Training model!


KeyboardInterrupt: 

### Count Encoding

Count encoding replaces each categorical value with the number of times it appears in the dataset.<br>
For example, if the value "GB" occured 10 times in the country feature, then each "GB" would be replaced with the number 10.

We'll use the categorical-encodings package to get this encoding. The encoder itself is available as CountEncoder.<br>
This encoder and the others in categorical-encodings work like scikit-learn transformers with .fit and .transform methods.

In [22]:
import category_encoders as ce
cat_features = ['category','currency','country']
count_enc = ce.CountEncoder()
count_encoded = count_enc.fit_transform(ks[cat_features])
count_encoded.head()

/opt/anaconda3/lib/python3.7/site-packages/category_encoders/count.py:255: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  X.loc[:, self.cols] = X.fillna(value=pd.np.nan)


,category,currency,country
0,1369,34132,33672
1,5188,295365,292627
2,5188,295365,292627
3,15727,295365,292627
4,10108,295365,292627


In [23]:
data = baseline_data.join(count_encoded.add_suffix("_count"))
data.head(3)

,goal,hour,day,month,year,outcome,category,currency,country,category_count,currency_count,country_count
0,1000.0,12,11,8,2015,0,108,5,9,1369,34132,33672
1,30000.0,4,2,9,2017,0,93,13,22,5188,295365,292627
2,45000.0,0,12,1,2013,0,93,13,22,5188,295365,292627


In [ ]:
# Training on the baseline data
train, valid, test = get_data_splits(data)
bst = train_model(train, valid)

### Target Encoding

Target encoding replaces a categorical value with the average value of the target for that value of the feature.
- For example, given the country value "CA", you'd calculate the average outcome for all the rows with country == 'CA', around 0.28.
- This is often blended with the target probability over the entire dataset to reduce the variance of values with few occurences.

This technique uses the targets to create new features. So including the validation or test data in the target encodings would be a form of target leakage.<br>
Instead,
you should learn the target encodings from the training dataset only and apply it to the other datasets.<br>
The category_encoders package provides TargetEncoder for target encoding. The implementation is similar to CountEncoder.

In [24]:
import category_encoders as ce
cat_features = ['category','currency','country']

# Create the encoder itself
target_enc = ce.TargetEncoder(cols=cat_features)
target_enc

TargetEncoder(cols=['category', 'currency', 'country'], drop_invariant=False,
              handle_missing='value', handle_unknown='value',
              min_samples_leaf=1, return_df=True, smoothing=1.0, verbose=0)

In [27]:
train, valid, _ = get_data_splits(data)

# Fit the encoder using the categorical features and target
target_enc.fit(train[cat_features], train['outcome'])
target_enc

TargetEncoder(cols=['category', 'currency', 'country'], drop_invariant=False,
              handle_missing='value', handle_unknown='value',
              min_samples_leaf=1, return_df=True, smoothing=1.0, verbose=0)

In [20]:
# Transform the features, rename the columns with _target suffix, and join to dataframe
train = train.join(target_enc.transform(train[cat_features]).add_suffix('_target'))
valid = valid.join(target_enc.transform(valid[cat_features]).add_suffix('_target'))
train.head(3)

,goal,hour,day,month,year,outcome,category,currency,country,category_count,currency_count,country_count,category_target,currency_target,country_target
0,1000.0,12,11,8,2015,0,108,5,9,1369,34132,33672,0.357815,0.354235,0.358672
1,30000.0,4,2,9,2017,0,93,13,22,5188,295365,292627,0.383693,0.371210,0.374410
2,45000.0,0,12,1,2013,0,93,13,22,5188,295365,292627,0.383693,0.371210,0.374410


In [21]:
bst = train_model(train, valid)

Training model!
Validation AUC score: 0.7525


### CatBoost Encoding

Finally, we'll look at CatBoost encoding. This is similar to target encoding in that it's based on the target probablity for a given value.<br>
However with CatBoost, for each row, the target probability is calculated only from the rows before it.

In [22]:
cat_features = ['category','currency','country']
target_enc = ce.CatBoostEncoder(cols=cat_features)

train, valud, _ = get_data_splits(data)
target_enc.fit(train[cat_features], train['outcome'])

train = train.join(target_enc.transform(train[cat_features]).add_suffix('_cb'))
valid = valid.join(target_enc.transform(valid[cat_features]).add_suffix('_cb'))

In [23]:
train.head()

,goal,hour,day,month,year,outcome,category,currency,country,category_count,currency_count,country_count,category_cb,currency_cb,country_cb
0,1000.0,12,11,8,2015,0,108,5,9,1369,34132,33672,0.357812,0.354235,0.358672
1,30000.0,4,2,9,2017,0,93,13,22,5188,295365,292627,0.383686,0.371210,0.374410
2,45000.0,0,12,1,2013,0,93,13,22,5188,295365,292627,0.383686,0.371210,0.374410
3,5000.0,3,17,3,2012,0,90,13,22,15727,295365,292627,0.410495,0.371210,0.374410
4,19500.0,8,4,7,2015,0,55,13,22,10108,295365,292627,0.300992,0.371210,0.374410


In [11]:
bst = train_model(train, valid)

Training model!
Validation AUC score: 0.7522
